In [1]:
# Importe les librairies utiles
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler 

# Chemin vers les csv
pathTest = "./test3.csv"
pathTrain = "./train3.csv"

# Importe les jeux de donnees
df_test = pd.read_csv(pathTest, sep=',', index_col=0)
df_train = pd.read_csv(pathTrain, sep=',', index_col=0)

In [2]:
"""

--- Variables Explicatives ---
Name : Nom du jeu
Platform : Console sur laquelle le jeu fonctionne
Year of release : Année de sortie du jeu
Genre
Publisher
JP_sales : Nombre de ventes du jeu au Japon en millions d’unités
Other sales : Nombre de ventes du jeu ailleurs dans le monde : Afrique, Asie sans le Japon, Europe sans l’Union Européenne et Amérique du Sud en millions d’unités
Critic_score : Score donné par Metacritic
Critic_count : Nombre de critiques prises en compte pour estimer le Critic_score
User_Score : Score donné par les usagers de Metacritic
User_Count : Nombre d’usagers considérés pour estimer le User_Score
Developer : Compagnie créatrice du jeu
Rating : Classement ESRB (Entertainment Software Rating Board) ie à qui s’addresse le jeu (tout public, majeur, adolescents, etc) 

--- Variable d'interet ---
NA_sales : Nombre de ventes du jeu en Amérique du Nord en millions d’unités
Global_Sales : Nombre de ventes total du jeu en millions d’unités

"""


"\n\n--- Variables Explicatives ---\nName : Nom du jeu\nPlatform : Console sur laquelle le jeu fonctionne\nYear of release : Année de sortie du jeu\nGenre\nPublisher\nJP_sales : Nombre de ventes du jeu au Japon en millions d’unités\nOther sales : Nombre de ventes du jeu ailleurs dans le monde : Afrique, Asie sans le Japon, Europe sans l’Union Européenne et Amérique du Sud en millions d’unités\nCritic_score : Score donné par Metacritic\nCritic_count : Nombre de critiques prises en compte pour estimer le Critic_score\nUser_Score : Score donné par les usagers de Metacritic\nUser_Count : Nombre d’usagers considérés pour estimer le User_Score\nDeveloper : Compagnie créatrice du jeu\nRating : Classement ESRB (Entertainment Software Rating Board) ie à qui s’addresse le jeu (tout public, majeur, adolescents, etc) \n\n--- Variable d'interet ---\nNA_sales : Nombre de ventes du jeu en Amérique du Nord en millions d’unités\nGlobal_Sales : Nombre de ventes total du jeu en millions d’unités\n\n"

In [3]:
"""
--- Evaluation des predictions --- 

Dans le document MTH3302_CriteresProjet-1.pdf on nous informe que la precision
de nos estimations sera evaluee avec le root mean square error (RMSE)

On definit cette fonction ci-bas,

    Y : Variable d'interet
    W : Predictions

"""

def RMSE(Y,W):
    
    # Nombre d'observations
    n = len(Y)
    
    total = 0.0
    for i in range(n):
        total += (Y[i] - W[i])**2
    
    mean = total/float(n)
    
    print("Mean Value = %.2f, RMSE = %.2f" % (np.mean(Y),mean))
    

In [4]:
"""
--- Variable Qualitative -> Table Binaire --- 

Afin de traiter nos variables qualitatives, nous les transformons en table binaire.
Nous nous basons sur la fonction get_dummies de la librairies Panda pour arriver a nos fins.

Input,
    trainSet : le dataframe complet contenant le jeu de donnees de training avec toutes ses colonnes
    testSet : le dataframe complet contenant le jeu de donnees de training avec toutes ses colonnes
    trainVar : la variable qualitative que nous souhaitons transformer en tableau de variable binaire
    testVar : la variable qualitative que nous souhaitons transformer en tableau de variable binaire

"""

def binerizedQualitativeVariable(trainSet, testSet, trainVar, testVar):
    
    # Generate Binary Table
    trainVar_binary = pd.get_dummies(trainVar, drop_first = True)
    testVar_binary = pd.get_dummies(testVar, drop_first = True)

    # We concatenate the new columns
    trainSet = pd.concat([trainSet,pd.DataFrame(trainVar_binary)],axis=1)
    testSet = pd.concat([testSet,pd.DataFrame(testVar_binary)],axis=1)

    # Get missing columns in the test set
    missing_categories = set(trainSet[:]) - set(testSet[:])

    # Add the missing columns in test set with default value equal to 0
    for c in missing_categories:
        testSet[c] = 0

    # Get missing columns in the train set
    missing_categories = set(testSet[:]) - set(trainSet[:])

    # Add the missing columns in train set with default value equal to 0
    for c in missing_categories:
        trainSet[c] = 0

    # Ensure the order of columns in the test and train sets are the same
    trainSet, testSet = trainSet.align(testSet, axis=1)
    
    # We return the two set
    return trainSet, testSet

In [5]:
# Réglons le problèmes des NaN pour les valeurs qualitatives

from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)

df_train["Year_of_Release"] = df_train["Year_of_Release"].apply(pd.to_numeric)
df_test["Year_of_Release"] = df_test["Year_of_Release"].apply(pd.to_numeric)

imputer = imputer.fit(df_train.ix[:,[2,7,8,9,10]])
df_train.ix[:,[2,7,8,9,10]] = imputer.transform(df_train.ix[:,[2,7,8,9,10]])

df_test.ix[:,[1,6,7,8,9]] = imputer.transform(df_test.ix[:,[1,6,7,8,9]])



/home/iryasu/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/iryasu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':
/home/iryasu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
/home/iryas

In [6]:
# Separons les variables explicatives qualitatives de celles quantitatives
df_train_quanti = df_train.drop(['Name','Platform','Genre','Publisher','Developer','Rating'], axis=1)
df_test_quanti = df_test.drop(['Platform','Genre','Publisher','Developer','Rating'], axis=1)

In [7]:
df_test.ix[:,2].isna().sum()

/home/iryasu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


0

In [8]:
# On peut ici enlever les variables quantitatives non probantes
#df_train_quanti = df_train_quanti.drop(['Year_of_Release'], axis=1)
#df_train_quanti = df_train_quanti.drop(['JP_Sales'], axis=1)
#df_train_quanti = df_train_quanti.drop(['Other_Sales'], axis=1)
#df_train_quanti = df_train_quanti.drop(['Critic_Score'], axis=1)
#df_train_quanti = df_train_quanti.drop(['Critic_Count'], axis=1)
#df_train_quanti = df_train_quanti.drop(['User_Score'], axis=1)
#df_train_quanti = df_train_quanti.drop(['User_Count'], axis=1)
#df_train_quanti = df_train_quanti.drop(['NA_Sales'], axis=1)
#df_train_quanti = df_train_quanti.drop(['Global_Sales'], axis=1)

#df_test_quanti = df_test_quanti.drop(['Year_of_Release'], axis=1)
#df_test_quanti = df_test_quanti.drop(['JP_Sales'], axis=1)
#df_test_quanti = df_test_quanti.drop(['Other_Sales'], axis=1)
#df_test_quanti = df_test_quanti.drop(['Critic_Score'], axis=1)
#df_test_quanti = df_test_quanti.drop(['Critic_Count'], axis=1)
#df_test_quanti = df_test_quanti.drop(['User_Score'], axis=1)
#df_test_quanti = df_test_quanti.drop(['User_Count'], axis=1)

In [9]:
# Variable Qualitative #1 : Platform
print("Nbr. of categories (train) : %d" %(df_train.Platform.nunique()))
print("Nbr. of categories (test) : %d" %(df_test.Platform.nunique()))

df_train_quanti, df_test_quanti = binerizedQualitativeVariable(df_train_quanti, df_test_quanti, df_train.Platform, df_test.Platform)

Nbr. of categories (train) : 30
Nbr. of categories (test) : 27


In [10]:
# Variable Qualitative #2 : Genre
print("Nbr. of categories (train) : %d" %(df_train.Genre.nunique()))
print("Nbr. of categories (test) : %d" %(df_test.Genre.nunique()))

df_train_quanti, df_test_quanti = binerizedQualitativeVariable(df_train_quanti, df_test_quanti, df_train.Genre, df_test.Genre)

Nbr. of categories (train) : 12
Nbr. of categories (test) : 12


In [11]:
# Variable Qualitative #3 : Publisher
print("Nbr. of categories (train) : %d" %(df_train.Publisher.nunique()))
print("Nbr. of categories (test) : %d" %(df_test.Publisher.nunique()))

df_train_quanti, df_test_quanti = binerizedQualitativeVariable(df_train_quanti, df_test_quanti, df_train.Publisher, df_test.Publisher)

Nbr. of categories (train) : 552
Nbr. of categories (test) : 280


In [18]:
# Variable Qualitative #4 : Developer
print("Nbr. of categories (train) : %d" %(df_train.Developer.nunique()))
print("Nbr. of categories (test) : %d" %(df_test.Developer.nunique()))

"""
Bon ici on a un probleme, si on ajoute les colonnes binaires generees par cette variable on ajoute 100mb
a notre csv. Ceci rend peu pratique le prototypage, pour l'instant on le laisse tomber. On essayera de ce
donner une raison rationnelle de le domper plus tard dans l'analyse. Hypothese, beaucoup de colinearite avec 
le Publisher.
"""
#df_train_quanti, df_test_quanti = binerizedQualitativeVariable(df_train_quanti, df_test_quanti, df_train.Developer, df_test.Developer)

Nbr. of categories (train) : 1593
Nbr. of categories (test) : 677


"\nBon ici on a un probleme, si on ajoute les colonnes binaires generees par cette variable on ajoute 100mb\na notre csv. Ceci rend peu pratique le prototypage, pour l'instant on le laisse tomber. On essayera de ce\ndonner une raison rationnelle de le domper plus tard dans l'analyse. Hypothese, beaucoup de colinearite avec \nle Publisher.\n"

In [12]:
# Variable Qualitative #5 : Rating
print("Nbr. of categories (train) : %d" %(df_train.Rating.nunique()))
print("Nbr. of categories (test) : %d" %(df_test.Rating.nunique()))

df_train_quanti, df_test_quanti = binerizedQualitativeVariable(df_train_quanti, df_test_quanti, df_train.Rating, df_test.Rating)

Nbr. of categories (train) : 8
Nbr. of categories (test) : 5


In [13]:
# On regarde le nombre de NaN par variables explicatives
print("Nombre de NaN sur %d observations \n" % (len(df_train_quanti)))

counter = 0;
for i in df_train_quanti.isna().sum():
    if(i > 0):
        print("%s : %d" % (df_train_quanti.columns[counter],i))
        
    counter += 1

Nombre de NaN sur 14094 observations 



In [14]:
# On garde seulement les observations ou nous avons toutes les donnees (a ameliorer plus tard)
df_train_quanti = df_train_quanti.dropna()

# On calcule le nombre d'observations restantes
print("\nApres clean-up, nombre d'observation : %d" % (len(df_train_quanti)))


Apres clean-up, nombre d'observation : 14094


In [15]:
# On enleve les colonnes qui ne comportent que des zeros
cols_only0 = (df_train_quanti != 0).any(axis=0)
df_train_quanti = df_train_quanti.loc[:,cols_only0]
df_test_quanti = df_test_quanti.loc[:,cols_only0]

# On enleve toutes les colonnes qui sont en double
duplicates = df_train_quanti.T.duplicated()
duplicates = ~duplicates
df_train_quanti = df_train_quanti.T[duplicates].T
df_test_quanti = df_test_quanti.T[duplicates].T


In [23]:
# On sauvegarde nos deux nouveaux jeux de donnees entierement quantitatif
#df_train_quanti.to_csv("train_quanti.csv")
#df_test_quanti.to_csv("test_quanti.csv")

In [16]:
def linear_regression(y,x):
    
    # Number of Explicative Variables
    k = 0
    try:
        k = x.shape[1]
    except IndexError:
        k = 1
        
    
    # The variance-covariance
    C = np.linalg.inv(np.dot(x.T,x))
    
    B = np.dot(np.dot(C,x.T),y)
    
    return B, np.dot(x,B)

In [17]:
def R2_adj(Y,W,p):

    SS_tot = 0.0
    SS_reg = 0.0
    SS_res = 0.0    
    
    y_S = np.sum(Y)/len(Y)

    for i in range(len(Y)):
        SS_tot += (Y[i] - y_S)**2
        SS_reg += (W[i] - y_S)**2
        SS_res += (Y[i] - W[i])**2
        
    
    n = len(Y)
    
    Radj = 1 - ((SS_res)/float(n-p))/((SS_tot)/(n-1))

    return Radj

In [18]:
"""
Ici on prend en input la variable d'interet et la matrice des variables explicatives. Avec une analyse en 
composante principale on obtient de nouveau coefficient pour notre regression lineaire.

"""

def pca(Y,X):
    
    X_std = StandardScaler().fit_transform(X)

    # The covariance matrix
    E = np.dot(X_std.T,X_std)/(nbrOfVar-1)

    # Eigenvalues and Eigenvectors
    eig_val, eig_vec = np.linalg.eig(E) #vector already normalized

    eig_val = eig_val.real # Remove imaginary part

    eig_val_contribution = eig_val/np.sum(eig_val) # Extract the explained variable for each eigenvalues
    
    rangeIndex = np.array(range(len(eig_val_contribution)))
    eig_val_contribution = np.array(([rangeIndex,eig_val_contribution])).T # Concatenate an index column
    
    eig_val_contribution = eig_val_contribution[(-eig_val_contribution[:,1]).argsort()] # Sort in descending order for col[1]

    #print("Explained variance : ")
    sumSofar = 0.0
    stopIndex = 0
    for i in range(len(eig_val)):
        sumSofar += eig_val_contribution[i,1]
        #print("%d. Eig %d = %.4f --- sum : %.4f" % (i+1,eig_val_contribution[i,0],eig_val_contribution[i,1],sumSofar))
        if(sumSofar >= 0.9999):
            stopIndex = i
            break

    indexKept = eig_val_contribution[:stopIndex,0]

    # Our reduced regressor
    K_vect = eig_vec[indexKept.astype(int)].T.real

    # Our principal components
    Z = np.dot(X_std,K_vect)

    # Our Regression Coefficiants
    B_partial = np.dot(np.linalg.inv(np.dot(Z.T,Z)),Z.T)
    B = np.dot(B_partial,Y)
    
    # We calculate the intercept
    uY = np.mean(Y)

    # We add the intercept
    Y_new = np.dot(Z,B) + uY
    
    # Clip to min 0
    Y_new = Y_new.clip(min=0)

    # New R2 and RMSE
    try:
        RMSE(Y,Y_new)
        print("R2_adj : %.2f" %(R2_adj(Y,Y_new,nbrOfVar)))
    except np.linalg.LinAlgError:
        print("NUL, hahaha determinant = 0")
        
    return Y_new, B, K_vect, uY

In [19]:
def pca_trained(X,B,K_vect,uY):
    
    X_std = StandardScaler().fit_transform(X)
    
    # Our principal components
    Z = np.dot(X_std,K_vect)
    
    # We add the intercept
    Y_new = np.dot(Z,B) + uY
    
    # Clip to min 0
    Y_new = Y_new.clip(min=0)

    return Y_new
    

In [20]:
# On definit notre variable d'interet
Y = np.array(df_train_quanti.NA_Sales)

# On definit notre vecteur de variables explicatives
X = np.array(df_train_quanti.drop(['NA_Sales','Global_Sales'], axis=1))
nbrOfVar = X.shape[1]

In [21]:
# On s'assure que nos matrices soient toutes dans la bonne position
print("Y Shape : ",Y.shape)
print("X Shape : ",X.shape)
print("Nbr de Variables : ",nbrOfVar)

Y Shape :  (14094,)
X Shape :  (14094, 605)
Nbr de Variables :  605


In [22]:
# We train our model
Y_new, B, K_vect, uY = pca(Y,X)

Mean Value = 0.27, RMSE = 0.32
R2_adj : 0.51


In [23]:
# On regarde le nombre de NaN par variables explicatives
print("Nombre de NaN sur %d observations \n" % (len(df_test_quanti)))

counter = 0;
for i in df_test_quanti.isna().sum():
    if(i > 0):
        print("%s : %d" % (df_test_quanti.columns[counter],i))
        
    counter += 1

Nombre de NaN sur 2490 observations 



In [24]:
def estimateColumn(trainSet,testSet,colName):
    
    # On definit notre variable d'interet
    Y = np.array(trainSet[colName])

    # On definit notre vecteur de variables explicatives
    X = np.array(trainSet.drop([colName], axis=1))
    nbrOfVar = X.shape[1]

    # We train our model
    [Y_new, B, K_vect, uY] = pca(Y,X)    
    
    nbrOfEstimate = 0
    testSet_minusCol = testSet.drop([colName], axis=1)
    for i, row in testSet.iterrows():
        if(np.isnan(testSet.loc[i,colName])):
            X = np.array(testSet_minusCol.iloc[1])
            X = X.reshape(X.shape[0],1).T
            y = pca_trained(X,B,K_vect,uY)
            y = y[0]
            testSet.loc[i,colName] = y 
            nbrOfEstimate +=1
    
    print(nbrOfEstimate)
    
    return testSet

In [25]:
trainSet = df_train_quanti.drop(['Critic_Score','User_Count','User_Score','Year_of_Release'], axis=1)
testSet = df_test_quanti.drop(['Critic_Score','User_Count','User_Score','Year_of_Release'], axis=1)

df_test_quanti.Critic_Count = estimateColumn(trainSet,testSet,'Critic_Count')

Mean Value = 26.36, RMSE = 120.53
R2_adj : 0.29
0


ValueError: Buffer has wrong number of dimensions (expected 1, got 0)

In [27]:
trainSet = df_train_quanti.drop(['User_Count','User_Score','Year_of_Release'], axis=1)
testSet = df_test_quanti.drop(['User_Count','User_Score','Year_of_Release'], axis=1)
df_test_quanti.Critic_Score = estimateColumn(trainSet,testSet,'Critic_Score')

Mean Value = 69.01, RMSE = 66.25
R2_adj : 0.28
0


ValueError: Buffer has wrong number of dimensions (expected 1, got 0)

In [28]:
trainSet = df_train_quanti.drop(['User_Score','Year_of_Release'], axis=1)
testSet = df_test_quanti.drop(['User_Score','Year_of_Release'], axis=1)
df_test_quanti.User_Count = estimateColumn(trainSet,testSet,'User_Count')

Mean Value = 157.46, RMSE = 89852.49
R2_adj : 0.27
0


ValueError: Buffer has wrong number of dimensions (expected 1, got 0)

In [29]:
trainSet = df_train_quanti.drop(['Year_of_Release'], axis=1)
testSet = df_test_quanti.drop(['Year_of_Release'], axis=1)
df_test_quanti.User_Score = estimateColumn(trainSet,testSet,'User_Score')

Mean Value = 7.14, RMSE = 0.71
R2_adj : 0.27
0


ValueError: Buffer has wrong number of dimensions (expected 1, got 0)

In [30]:
trainSet = df_train_quanti
testSet = df_test_quanti
df_test_quanti.Year_of_Release = estimateColumn(trainSet,testSet,'Year_of_Release')

Mean Value = 2006.45, RMSE = 5.05
R2_adj : 0.84
0


ValueError: Buffer has wrong number of dimensions (expected 1, got 0)

In [31]:
# On regarde le nombre de NaN par variables explicatives
print("Nombre de NaN sur %d observations \n" % (len(df_test_quanti)))

counter = 0;
for i in df_test_quanti.isna().sum():
    if(i > 0):
        print("%s : %d" % (df_test_quanti.columns[counter],i))
        
    counter += 1

Nombre de NaN sur 2490 observations 



In [32]:
def addColumn(trainSet,testSet,colName):
    
    # On definit notre variable d'interet
    Y = np.array(trainSet[colName])

    # On definit notre vecteur de variables explicatives
    X = np.array(trainSet.drop([colName], axis=1))

    # We train our model
    [Y_new, B, K_vect, uY] = pca(Y,X)
    
    
    X_test = np.array(testSet.drop([colName], axis=1))    
    Y_test = pca_trained(X_test,B,K_vect,uY)
    
    return Y_test

In [33]:
df_test_quanti['NA_Sales'] = addColumn(df_train_quanti,df_test_quanti,'NA_Sales')

Mean Value = 0.27, RMSE = 0.04
R2_adj : 0.94


ValueError: Buffer has wrong number of dimensions (expected 1, got 0)

In [34]:
df_test_quanti['Global_Sales'] = addColumn(df_train_quanti,df_test_quanti,'Global_Sales')

Mean Value = 0.54, RMSE = 0.09
R2_adj : 0.96


ValueError: Buffer has wrong number of dimensions (expected 1, got 0)

In [35]:
df_test_final = df_test_quanti[['Global_Sales','NA_Sales']]

In [36]:
df_test_final.to_csv("test_final.csv")